In [155]:
import matplotlib.pyplot as plt 

import scipy.ndimage.filters

from skimage.transform import pyramid_laplacian

from PIL import Image
from PIL import ImageFilter

import numpy as np

class pyramid(object):
    g_data=[]
    l_data=[]
    
    def __init__(self,file_path,n=5):
        self.g_data=[]
        self.l_data=[]
        im = Image.open(file_path)
        prev=np.asarray(im,dtype=np.float64)/255.0
#         print(prev.shape)
        self.g_data.append(prev)
        
#         plt.imshow(prev, cmap="Greys_r")
#         plt.imshow(np.subtract(prev,self._blur(prev)),cmap="Greys_r")
        for i in range(n):
            the_blur=pyramid.blur(prev)
            diff=prev-the_blur
            self.l_data.append(diff)
            smaller=pyramid.subsample(the_blur)
            self.g_data.append(smaller)
            prev=smaller
            
        self.l_data.append(self.g_data[-1])
    
    
# A wrong way of making laplacian pyramids
#         rev =list(reversed(self.g_data))
#         for i in range(n):              
#             expanded=scipy.ndimage.interpolation.zoom(rev[i],2,order=0)
# #             print expanded.shape
#             expanded=self._blur(expanded)
#             diff=rev[i+1]-expanded
#             self.l_data.append(diff)
            
            
    @staticmethod
    def blur(img_arr):
        return scipy.ndimage.filters.gaussian_filter(img_arr, 1)
    
    @staticmethod
    def subsample(img_arr):
        sub_arr = img_arr[::2,::2]
        return sub_arr    
    
    @staticmethod
    def supersample(img_arr):
        new_arr=scipy.ndimage.interpolation.zoom(img_arr,2,order=0)
        for x in range(new_arr.shape[0]):
            if x%2==1:
                new_arr[:,x]=np.zeros(new_arr.shape[1])
        for y in range(new_arr.shape[1]):
            if y%2==1:
                new_arr[y,:]=np.zeros(new_arr.shape[0])
        return new_arr
      
    @staticmethod
    def reconstruct(lapl):
        rev_lapl =list(reversed(lapl))
        gauss=lapl[-1]
        for i in range(len(lapl)-1):
            up=pyramid.supersample(gauss)
            blurred=pyramid.blur(up)
            added=blurred+rev_lapl[i+1]
            gauss=added
            plt.figure()
            plt.imshow(gauss,cmap="Greys_r")
            
#         plt.imshow(self._blur(self._supersample(lapl[-1])+lapl[-2]),cmap="Greys_r")
        
    def level(self,i):
        return self.g_data[i]
        
    def show(self,source):
        for i in range(len(source)):
            plt.figure()
            plt.imshow(source[i], cmap="Greys_r")
            
    def show_g(self):
        self.show(self.g_data)
        
    def show_l(self):
        self.show(self.l_data)

        

        
        
        
        

In [158]:
%matplotlib inline
p=pyramid("Images/Q1/toucan.tif")
# p.show_g()
# p.show_l()

laplacian=p.l_data
# pyramid.reconstruct(laplacian)

# print(p.l_data[0])
# x=np.array([[1,2],[3,4]])
# print(p._supersample(x))
    
# p._supersample(x)




# image=np.asarray(Image.open("Images/Q1/mandril.tif"),dtype=np.float64)/255.0
# # plt.imshow(image,cmap="Greys_r")
# thing=pyramid_laplacian(image, max_layer=-1, downscale=2, sigma=2, order=1, mode='reflect', cval=0)
# for x in thing:
#     print(x[0][0])
# #     plt.figure()
# #     plt.imshow(x, cmap="Greys_r")
#     break
print(p.l_data[-1])
# print(p.l2_data[0][0][0])
# print np.asarray(Image.open("Images/Q1/lena_gray.gif"))
# p=g_pyramid("Images/Q1/lena_gray.gif")
# p.show_all()
# p.show_l()
# p.show_g()
# plt.imshow(p.level(-1), cmap="Greys_r")


[[  4.30712593e-01   4.38892000e-01   5.88322717e-01   6.49468250e-01
    5.49057680e-01   5.22872867e-01   5.04063254e-01   5.40520321e-01]
 [  3.73370677e-01   4.42891629e-01   5.40480780e-01   5.66913297e-01
    5.41676022e-01   4.49058780e-01   3.60175353e-01   4.77046856e-01]
 [  2.56503617e-01   3.62096550e-01   4.43947667e-01   5.14388485e-01
    5.01637034e-01   5.07518851e-01   4.21698799e-01   3.91807314e-01]
 [  2.65143939e-01   3.28338095e-01   3.87399680e-01   5.37077464e-01
    5.04611664e-01   5.61357793e-01   4.36420915e-01   3.16944539e-01]
 [  3.28174126e-01   3.24559217e-01   3.78424145e-01   5.23476759e-01
    5.34009629e-01   5.73779669e-01   3.86780148e-01   1.56590274e-01]
 [  5.34483332e-01   5.06188483e-01   4.89958375e-01   4.90629044e-01
    3.83892290e-01   4.10403377e-01   2.34766630e-01   3.98766565e-02]
 [  6.49812761e-01   6.56732890e-01   6.47825365e-01   5.21162531e-01
    1.83009211e-01   8.72768643e-02   3.75854487e-02   3.49901955e-03]
 [  7.0834066